## **Computer vision project: Deggendorf Waste Sorting Assistant**

### **Overview**
The Deggendorf Waste Sorting Assistant is a Computer Vision-based tool designed to help residents and international students correctly identify waste bins. The project leverages image classification to determine the category of a given waste bin based on its visual characteristics. Users can take a picture of an unlabeled bin, and the model will classify it while providing information on the appropriate waste materials for disposal.

### **Project Goals**
- Develop an image classification model capable of identifying waste bins in Deggendorf.
- Provide users with clear guidance on proper waste disposal based on bin classification.
- Document all processes in a Jupyter Notebook, covering dataset creation, model training, evaluation, and deployment.


### 1. Mount Google Drive & Interactive Labeling Utility

This section sets up everything you need to label images **in-Colab**:

1. Installs required packages  
2. Mounts your Drive  
3. Enables Colab’s custom widget manager for `ipywidgets`  
4. Defines constants, logging, and a CSV to track labels  
5. Provides an interactive widget UI to:
   - Scan `/MyDrive/cv_garbage` for unlabeled images  
   - Display one image at a time  
   - Pick a label from a fixed list  
   - Copy the image into `/MyDrive/cv_garbage/labled` with a standardized name  
   - Record `original_filename`, `new_filename`, `label`, and `timestamp` in `labels.csv`  

---

In [ ]:
# 1.0 Install dependencies
%pip install --quiet --upgrade pandas==2.2.2 pillow ipywidgets

In [ ]:
# 1.1 Mount Drive & enable widgets
from google.colab import drive, output
drive.mount('/content/drive', force_remount=True)
output.enable_custom_widget_manager()


In [ ]:
# 1.2 Imports, Paths & Logging
from pathlib import Path
import shutil, logging, pandas as pd
from datetime import datetime
from IPython.display import display, clear_output
from PIL import Image as PILImage
import ipywidgets as widgets

BASE_DIR    = Path('/content/drive/MyDrive/cv_garbage')
LABELED_DIR = BASE_DIR / 'labeled'
CSV_PATH    = LABELED_DIR / 'labels.csv'
LABELS      = ["Restmüll", "Biomüll", "Papier", "Gelber_Sack", "Glas"]

LABELED_DIR.mkdir(parents=True, exist_ok=True)

logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(levelname)s: %(message)s",
                    force=True)

In [ ]:
# 1.3 Labeler Class Definition
class ColabImageLabeler:
    """Interactive image labeling helper for Google Colab."""
    EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp'}

    def __init__(self, src_dir: Path, dst_dir: Path, csv_path: Path, labels: list[str]):
        self.src_dir   = Path(src_dir)
        self.dst_dir   = Path(dst_dir)
        self.csv_path  = Path(csv_path)
        self.labels    = labels

        self._df       = self._load_or_init_csv()
        self._images   = self._scan_for_unlabeled()
        self._index    = 0

        # --- Widgets -------------------------------------------------------------------
        self._out      = widgets.Output(layout={'border': '1px solid #ccc'})
        self._w_label  = widgets.Select(
            options=self.labels,
            value=self.labels[0],
            description='Label:',
            rows=len(self.labels)
        )
        self._w_next   = widgets.Button(description='Save & Next ▶️', button_style='success')
        self._w_skip   = widgets.Button(description='Skip ⏭️')
        self._w_status = widgets.HTML()

        self._w_next.on_click(self._on_save_next)
        self._w_skip.on_click(self._on_skip)

    # ----------------------------------------------------------------------------------
    # Data helpers
    # ----------------------------------------------------------------------------------
    def _load_or_init_csv(self) -> pd.DataFrame:
        if self.csv_path.exists():
            logging.info("Found existing label file: %s", self.csv_path)
            return pd.read_csv(self.csv_path)
        else:
            logging.info("Creating new label file: %s", self.csv_path)
            return pd.DataFrame(columns=['original_filename', 'new_filename', 'label', 'timestamp'])

    def _scan_for_unlabeled(self) -> list[str]:
        all_imgs = sorted([p.name for p in self.src_dir.iterdir()
                           if p.suffix.lower() in self.EXTENSIONS])
        already  = set(self._df['original_filename'])
        unlabeled = [f for f in all_imgs if f not in already]
        logging.info("Unlabeled images: %d", len(unlabeled))
        return unlabeled

    # ----------------------------------------------------------------------------------
    # UI actions
    # ----------------------------------------------------------------------------------
    def _display_current_image(self):
        with self._out:
            clear_output(wait=True)
            img_path = self.src_dir / self._images[self._index]
            try:
                img = PILImage.open(img_path)
                img.thumbnail((640, 480))
                display(img)
            except Exception as ex:
                logging.error("Error displaying %s: %s", img_path, ex)
                display(widgets.HTML(f"<b>Error loading image {img_path.name}</b>"))

    def _update_status(self, message: str, level: str = 'info'):
        colors = {'info': '#333', 'warn': 'orange', 'error': 'red', 'success': 'green'}
        self._w_status.value = f"<span style='color:{colors.get(level,'#333')}'>{message}</span>"

    def _persist_label(self, orig_name: str, new_name: str, label: str):
        ts = datetime.utcnow().isoformat(timespec='seconds') + 'Z'
        self._df.loc[len(self._df)] = [orig_name, new_name, label, ts]
        self._df.to_csv(self.csv_path, index=False)

    def _advance(self):
        if self._index >= len(self._images):
            with self._out:
                clear_output(wait=True)
            self._update_status("🎉 All images processed!", 'success')
            self._w_next.disabled = self._w_skip.disabled = True
            return False
        self._display_current_image()
        self._update_status(f"Image {self._index+1}/{len(self._images)} · {self._images[self._index]}", 'info')
        return True

    def _on_save_next(self, _):
        orig_name = self._images[self._index]
        label     = self._w_label.value.replace(' ', '_')
        extension = Path(orig_name).suffix
        new_name  = f"{label}_{len(self._df)+1:05d}{extension}"
        try:
            shutil.copy2(self.src_dir / orig_name, self.dst_dir / new_name)
            self._persist_label(orig_name, new_name, label)
            self._update_status(f"✔️ {orig_name} → {new_name}", 'success')
        except Exception as ex:
            logging.error("Failed to copy %s: %s", orig_name, ex)
            self._update_status(f"❌ Error: {ex}", 'error')
            return
        self._index += 1
        self._advance()

    def _on_skip(self, _):
        self._index += 1
        self._update_status("⏭️ Skipped.", 'warn')
        self._advance()

    # ----------------------------------------------------------------------------------
    # Public API
    # ----------------------------------------------------------------------------------
    def start(self):
        if not self._images:
            self._update_status("Nothing to label in ‘%s’." % self.src_dir, 'warn')
            display(self._w_status)
            return

        self._advance()
        ui = widgets.VBox([
            self._out,
            widgets.HBox([self._w_label, self._w_next, self._w_skip]),
            self._w_status
        ])
        display(ui)

In [ ]:
# 1.4 Launch Labeler
labeler = ColabImageLabeler(
    src_dir=BASE_DIR,
    dst_dir=LABELED_DIR,
    csv_path=CSV_PATH,
    labels=LABELS
)
labeler.start()

### 2. Import Required Libraries for the Rest of the Project

In [ ]:
# 1. Import Required Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 3. Dataset Creation

In [ ]:
# TODO: code for splitting data, creating train/val folders, etc. …

### 4. Model Training

In [ ]:
# TODO: code for defining and training your CNN …

### 5. Evaluation & Deployment

In [ ]:
# TODO: code for evaluating accuracy, exporting a TensorFlow Lite model, etc. …